Cross Validation 5

size =  121655

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.0) & (df['Maximum_Installs'] >= 5000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_train_5.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_val_5.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(97324, 23)
(24331, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Scorpion in phone prank,us.funset.scorpioonscreen,Entertainment,4.3,49161,False,Do you want to scare a friend? Put the scorpio...,https://justpaste.it/17e7f,0,0,...,0,0,1,0,0,0,663,do you want to scare a friend? put the scorpio...,644,135
1,Daily Connect (Child Care),com.seacloud.dc,Education,4.5,57235,False,Daily Connect makes it easier than ever to run...,https://www.dailyconnect.com/privacy,0,1,...,0,0,1,0,0,0,3608,daily connect makes it easier than ever to run...,3365,622


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera        16414
Location      16300
Microphone     8907
Contacts      10592
Storage       48146
Phone         15140
SMS             344
Call_Log        217
Calendar       1779
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera         4039
Location       4042
Microphone     2134
Contacts       2528
Storage       12112
Phone          3737
SMS              69
Call_Log         49
Calendar        462
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

121655


In [23]:
corpus[0]

['want',
 'scare',
 'friend',
 'put',
 'scorpion',
 'inside',
 'friends',
 'phone',
 'realistic',
 'animation',
 'walking',
 'scorpion',
 'displayed',
 'first',
 'row',
 'phone',
 'screen',
 'surprising',
 'scorpion',
 'shows',
 'selected',
 'time',
 'choose',
 'appear',
 'animation',
 'realistic',
 'scare',
 'everyone',
 'afraid',
 'arachnids',
 'main',
 'features',
 'scorpion',
 'phone',
 'realistic',
 'animation',
 'walking',
 'arachnid',
 'choose',
 'appear',
 'screen',
 'hide',
 'scorpio',
 'clicking',
 'notification',
 'choose',
 'transparency',
 'animation',
 'victim',
 'normally',
 'use',
 'phone',
 'animal',
 'moving']

##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        want scare friend put scorpion inside friends ...
1        daily connect makes easier ever run childcare ...
2        bank conveniently securely farmers national ba...
3        ardex americas product calculator information ...
4        xbox fans gain access exclusive content events...
                               ...                        
97319    stylist wallpapers keyboard beautiful keyboard...
97320    consists precise easy understand meanings phys...
97321    may light guide dark places pendulum artifact ...
97322    leo browser made minimalism comfortable web br...
97323    boltnet vpn protects internet session helps ac...
Name: Clean_Description, Length: 97324, dtype: object

In [29]:
train_sequences

[[39,
  7710,
  883,
  729,
  13529,
  907,
  49,
  7,
  1472,
  954,
  2802,
  13529,
  1057,
  156,
  3066,
  7,
  15,
  7462,
  13529,
  490,
  576,
  13,
  65,
  1322,
  954,
  1472,
  7710,
  486,
  5749,
  58057,
  309,
  9,
  13529,
  7,
  1472,
  954,
  2802,
  34114,
  65,
  1322,
  15,
  609,
  5851,
  1677,
  287,
  65,
  3370,
  954,
  11140,
  4200,
  3,
  7,
  695,
  1029],
 [86,
  313,
  324,
  463,
  500,
  632,
  11902,
  1036,
  529,
  429,
  201,
  45,
  9027,
  622,
  11902,
  1804,
  800,
  1074,
  657,
  638,
  11902,
  3518,
  319,
  92,
  40,
  86,
  313,
  106,
  117,
  13,
  387,
  792,
  7352,
  1480,
  73,
  13,
  187,
  504,
  53,
  391,
  626,
  237,
  106,
  86,
  300,
  1992,
  792,
  1117,
  11902,
  1036,
  4967,
  5047,
  564,
  4864,
  11902,
  1466,
  319,
  92,
  3,
  86,
  313,
  342,
  11902,
  1036,
  3174,
  902,
  710,
  70,
  1557,
  181,
  43,
  792,
  1474,
  33,
  135,
  4997,
  800,
  1,
  308,
  3546,
  11902,
  3518,
  86,
  313,
  16,


In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[  39, 7710,  883, ...,    0,    0,    0],
       [  86,  313,  324, ...,    0,    0,    0],
       [ 212, 1448, 1570, ...,    0,    0,    0],
       ...,
       [ 167,  288,  480, ...,    0,    0,    0],
       [4414,  621,  260, ...,    0,    0,    0],
       [ 221, 3159,   81, ...,    0,    0,    0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[  583,  2988,  6628, ...,     0,     0,     0],
       [  269,  7070,  3347, ...,     0,     0,     0],
       [  149,    16,   692, ...,     0,     0,     0],
       ...,
       [  863, 13643,  1109, ...,     0,     0,     0],
       [   31,  3086,  8599, ...,     0,     0,     0],
       [  406,  3583,   790, ...,     0,     0,     0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

want scare friend put scorpion inside friends phone realistic animation walking scorpion displayed first row phone screen surprising scorpion shows selected time choose appear animation realistic scare everyone afraid arachnids main features scorpion phone realistic animation walking arachnid choose appear screen hide scorpio clicking notification choose transparency animation victim normally use phone animal moving
[39, 7710, 883, 729, 13529, 907, 49, 7, 1472, 954, 2802, 13529, 1057, 156, 3066, 7, 15, 7462, 13529, 490, 576, 13, 65, 1322, 954, 1472, 7710, 486, 5749, 58057, 309, 9, 13529, 7, 1472, 954, 2802, 34114, 65, 1322, 15, 609, 5851, 1677, 287, 65, 3370, 954, 11140, 4200, 3, 7, 695, 1029]


In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 194462


In [36]:
word_index

{'app': 1,
 'free': 2,
 'use': 3,
 'wallpaper': 4,
 'keyboard': 5,
 'theme': 6,
 'phone': 7,
 'application': 8,
 'features': 9,
 'new': 10,
 'get': 11,
 'wallpapers': 12,
 'time': 13,
 'also': 14,
 'screen': 15,
 'best': 16,
 'one': 17,
 'like': 18,
 'download': 19,
 'make': 20,
 'easy': 21,
 'android': 22,
 'launcher': 23,
 'us': 24,
 'english': 25,
 'live': 26,
 'share': 27,
 'mobile': 28,
 'find': 29,
 'help': 30,
 'learn': 31,
 'set': 32,
 'love': 33,
 'hd': 34,
 'device': 35,
 'please': 36,
 'apps': 37,
 'video': 38,
 'want': 39,
 'using': 40,
 'need': 41,
 'images': 42,
 'information': 43,
 'support': 44,
 'home': 45,
 'simple': 46,
 'themes': 47,
 'many': 48,
 'friends': 49,
 'available': 50,
 'text': 51,
 'enjoy': 52,
 'different': 53,
 'data': 54,
 'day': 55,
 'language': 56,
 'access': 57,
 'beautiful': 58,
 'game': 59,
 'add': 60,
 'easily': 61,
 'online': 62,
 'user': 63,
 'way': 64,
 'choose': 65,
 'you': 66,
 'words': 67,
 'create': 68,
 'play': 69,
 'every': 70,
 'games'

In [37]:
word_index["reason"]

2395

In [38]:
print(validation_sequences[0])

[583, 2988, 6628, 6454, 92, 31, 583, 97, 472, 71, 583, 71, 430, 4903, 2193, 6628, 3332, 396, 583, 1432, 26521, 1893, 21970, 3775, 2340, 8514, 626, 40, 625, 583, 71, 2800, 191, 76, 531, 3019, 6009, 377, 441, 3546, 31, 441, 3546, 441, 336, 1027, 4395, 1727, 1641, 7979, 1766, 31, 46, 441, 6687, 1027, 4853, 8280, 4281, 13132, 5731, 1027, 82, 407, 6194, 514, 1375, 31, 8280, 2345, 4281, 3142, 19814, 4281, 1806, 12832, 6478, 69, 441, 6687, 19439, 500, 6503, 3283, 143, 12556, 47857, 47857, 47857, 22145, 319, 1886, 2142, 154, 31, 391, 1641, 4058, 7979, 18, 10689, 5165, 14928, 22670, 41203, 1183, 361, 298, 2248, 31, 1641, 527, 1397, 406, 1014, 3, 413, 441, 703, 178, 1183, 41, 406, 1014, 385, 9003, 521, 148, 92, 583, 31, 53, 5940, 263, 24, 766, 92, 583, 173, 6628, 775, 583, 76, 71, 92, 97, 263, 24, 766, 92, 3451, 583, 688, 1177, 71, 97, 1799, 1399, 5940, 583, 1593, 626, 583, 9253, 31, 476, 148, 92, 583, 2152, 97, 583, 71, 152, 1799, 71, 2, 626, 685, 154]


In [39]:
word_index["listen"]

394

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.30414   , -0.37029999, -0.15881   , ..., -0.37830999,
        -0.42910001,  0.0030023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.62230998, -0.40289   ,  0.11949   , ...,  0.93154001,
         0.22446001,  0.15016   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
word_index["reason"]

2395

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(97324, 600)
(97324, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(24331, 600)
(24331, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

97324
9
{0: 0.6588142913231253, 1: 0.6634219495569189, 2: 1.2140763194990207, 3: 1.0209382343068143, 4: 0.22460386694175585, 5: 0.7142521649787171, 6: 31.435400516795866, 7: 49.833077316948284, 8: 6.078570982449566}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    58338900    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    58338900    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    58338900    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
3042/3042 [==============================] - 602s 194ms/step - loss: 0.1598 - accuracy: 0.2722 - val_loss: 0.2582 - val_accuracy: 0.2751
Epoch 2/10
3042/3042 [==============================] - 548s 180ms/step - loss: 0.1307 - accuracy: 0.2939 - val_loss: 0.2146 - val_accuracy: 0.2976
Epoch 3/10
3042/3042 [==============================] - 586s 193ms/step - loss: 0.1109 - accuracy: 0.3018 - val_loss: 0.2107 - val_accuracy: 0.2917
Epoch 4/10
3042/3042 [==============================] - 548s 180ms/step - loss: 0.0908 - accuracy: 0.3042 - val_loss: 0.2236 - val_accuracy: 0.3037
Epoch 5/10
3042/3042 [==============================] - 549s 180ms/step - loss: 0.0769 - accuracy: 0.3110 - val_loss: 0.2309 - val_accuracy: 0.3385
Epoch 6/10
3042/3042 [==============================] - 589s 194ms/step - loss: 0.0689 - accuracy: 0.3187 - val_loss: 0.2521 - val_accuracy: 0.2922
Epoch 7/10
3042/3042 [==============================] - 551s 181ms/step - loss: 0.0606 - accuracy: 0.3331 - val_

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2742, 1967, 29, 16, 2742, 9488, 416, 684, 2331, 1, 1, 195, 13, 16, 416, 2742, 1012, 416, 9488, 141, 416, 2742, 124, 16, 416, 2742, 1012, 416, 9488, 18, 27893, 38729, 13714, 14550, 34390, 331, 14, 195, 584, 416, 2742, 1652, 416, 2742, 416, 10414, 141, 416, 676, 416, 676, 1, 195, 16, 500, 27

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[3.08942166e-04, 1.04190223e-02, 1.25731342e-04, 1.04404916e-03,
        9.09953833e-01, 1.42205600e-02, 6.09618169e-08, 2.72460721e-09,
        1.89386284e-07],
       [8.59455668e-07, 3.66034694e-02, 1.35896407e-05, 1.55409763e-03,
        6.84523061e-02, 7.38746822e-02, 1.20977693e-05, 3.82287681e-07,
        1.74189836e-06],
       [1.17139971e-04, 1.24026046e-04, 1.89281072e-05, 1.06761395e-03,
        1.14735393e-02, 1.54598383e-02, 2.50064768e-05, 3.53847412e-07,
        1.36084063e-05],
       [1.04305821e-06, 1.72741624e-04, 6.44665033e-06, 5.50987478e-03,
        1.24824671e-02, 3.68996453e-03, 5.49598772e-05, 1.11208226e-06,
        4.31681488e-04],
       [4.84311245e-02, 2.53726874e-04, 7.13312021e-03, 7.64807835e-02,
        2.01163962e-01, 5.17705441e-01, 2.53632315e-04, 1.10561086e-04,
        3.81118560e-04],
       [4.83962446e-02, 1.32951457e-02, 6.29372383e-03, 1.74940284e-02,
        9.67888892e-01, 1.59852922e-01, 2.10513917e-06, 4.80073140e-07,
        4.4

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[3.08942166e-04 1.04190223e-02 1.25731342e-04 ... 6.09618169e-08
  2.72460721e-09 1.89386284e-07]
 [8.59455668e-07 3.66034694e-02 1.35896407e-05 ... 1.20977693e-05
  3.82287681e-07 1.74189836e-06]
 [1.17139971e-04 1.24026046e-04 1.89281072e-05 ... 2.50064768e-05
  3.53847412e-07 1.36084063e-05]
 ...
 [8.57617855e-01 7.87368774e-01 4.15475070e-01 ... 1.51164671e-02
  1.47884367e-02 5.25657237e-02]
 [5.56654140e-06 9.95160758e-01 9.99871016e-01 ... 1.07382255e-14
  1.26439395e-14 4.09793756e-06]
 [1.39295225e-05 1.14791306e-04 2.34779189e-04 ... 5.61210868e-07
  8.07832823e-09 1.18308117e-05]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.0432 with F-Score: 0.6843
permission  1
Length of sequence: 10000
Best Threshold: 0.4743 with F-Score: 0.5719
permission  2
Length of sequence: 10000
Best Threshold: 0.573 with F-Score: 0.5914
permission  3
Length of sequence: 10000
Best Threshold: 0.2704 with F-Score: 0.5365
permission  4
Length of sequence: 10000
Best Threshold: 0.0907 with F-Score: 0.7984
permission  5
Length of sequence: 10000
Best Threshold: 0.1809 with F-Score: 0.5225
permission  6
Length of sequence: 10000
Best Threshold: 0.1012 with F-Score: 0.0851
permission  7
Length of sequence: 10000
Best Threshold: 0.161 with F-Score: 0.3478
permission  8
Length of sequence: 10000
Best Threshold: 0.4819 with F-Score: 0.3485
-------------------------------------
optimal threshold tuning for f-score
[0.0432, 0.4743, 0.573, 0.2704, 0.0907, 0.1809, 0.1012, 0.161, 0.4819]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.004699999932199717 with G-Mean: 0.8383
FPR: 0.1374, TPR: 0.8148
permission  1
Best Threshold: 0.02630000002682209 with G-Mean: 0.7613
FPR: 0.2441, TPR: 0.7668
permission  2
Best Threshold: 0.01209999993443489 with G-Mean: 0.8147
FPR: 0.1514, TPR: 0.7821
permission  3
Best Threshold: 0.03139999881386757 with G-Mean: 0.8034
FPR: 0.1933, TPR: 0.8
permission  4
Best Threshold: 0.1324000060558319 with G-Mean: 0.7848
FPR: 0.2259, TPR: 0.7956
permission  5
Best Threshold: 0.013899999670684338 with G-Mean: 0.7596
FPR: 0.3046, TPR: 0.8298
permission  6
Best Threshold: 9.999999747378752e-05 with G-Mean: 0.8171
FPR: 0.2656, TPR: 0.9091
permission  7
Best Threshold: 0.00039999998989515007 with G-Mean: 0.9399
FPR: 0.1165, TPR: 1.0
permission  8
Best Threshold: 0.007499999832361937 with G-Mean: 0.828
FPR: 0.131, TPR: 0.7889
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0047, 0.0263, 0.0121, 0.0314, 0.1324, 0.0139, 1e-04, 0.0004, 0.007

## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV5_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6843
  Location  : 0.5719
  Microphone: 0.5914
  Contacts  : 0.5365
  Storage   : 0.7984
  Phone     : 0.5225
  SMS       : 0.0851
  Call_Log  : 0.3478
  Calendar  : 0.3485

  Average F1 score: 0.4985


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV5_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8942
  Location  : 0.8782
  Microphone: 0.9304
  Contacts  : 0.8819
  Storage   : 0.7829
  Phone     : 0.8534
  SMS       : 0.9907
  Call_Log  : 0.9968
  Calendar  : 0.9814

  Average F1 (micro) score: 0.9100


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV5_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8380
  Location  : 0.7613
  Microphone: 0.8142
  Contacts  : 0.8024
  Storage   : 0.7848
  Phone     : 0.7618
  SMS       : 0.8148
  Call_Log  : 0.9407
  Calendar  : 0.8234

  Average ROC_AUC score: 0.8157


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.0768 with F-Score: 0.4417
permission  1
Length of sequence: 10000
Best Threshold: 0.281 with F-Score: 0.5482
permission  2
Length of sequence: 10000
Best Threshold: 0.7452 with F-Score: 0.5588
permission  3
Length of sequence: 10000
Best Threshold: 0.1185 with F-Score: 0.596
permission  4
Length of sequence: 10000
Best Threshold: 0.0026 with F-Score: 0.5847
permission  5
Length of sequence: 10000
Best Threshold: 0.8628 with F-Score: 0.4175
permission  6
Length of sequence: 10000
Best Threshold: 0.023 with F-Score: 0.5323
permission  7
Length of sequence: 10000
Best Threshold: 0.0945 with F-Score: 0.4024
permission  8
Length of sequence: 10000
Best Threshold: 0.398 with F-Score: 0.5644
-------------------------------------
optimal threshold tuning for f-score
[0.0768, 0.281, 0.7452, 0.1185, 0.0026, 0.8628, 0.023, 0.0945, 0.398]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.0024999999441206455 with G-Mean: 0.7033
FPR: 0.3637, TPR: 0.7773
permission  1
Best Threshold: 0.10050000250339508 with G-Mean: 0.7207
FPR: 0.2919, TPR: 0.7336
permission  2
Best Threshold: 0.03920000046491623 with G-Mean: 0.7908
FPR: 0.1951, TPR: 0.777
permission  3
Best Threshold: 0.12080000340938568 with G-Mean: 0.7057
FPR: 0.2656, TPR: 0.6782
permission  4
Best Threshold: 0.21709999442100525 with G-Mean: 0.57
FPR: 0.4659, TPR: 0.6083
permission  5
Best Threshold: 0.16359999775886536 with G-Mean: 0.7754
FPR: 0.2708, TPR: 0.8246
permission  6
Best Threshold: 0.0007999999797903001 with G-Mean: 0.7574
FPR: 0.2845, TPR: 0.8019
permission  7
Best Threshold: 0.0005000000237487257 with G-Mean: 0.7598
FPR: 0.261, TPR: 0.7812
permission  8
Best Threshold: 0.021700000390410423 with G-Mean: 0.8243
FPR: 0.1687, TPR: 0.8173
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0025, 0.1005, 0.0392, 0.1208, 0.2171, 0.1636, 0.0008, 0.0005, 

#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV5_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4417
  Location  : 0.5482
  Microphone: 0.5588
  Contacts  : 0.5960
  Storage   : 0.5847
  Phone     : 0.4175
  SMS       : 0.5323
  Call_Log  : 0.4024
  Calendar  : 0.5644

  Average F1 score: 0.5162


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV5_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8069
  Location  : 0.7878
  Microphone: 0.9151
  Contacts  : 0.7171
  Storage   : 0.4264
  Phone     : 0.9151
  SMS       : 0.8670
  Call_Log  : 0.9286
  Calendar  : 0.9498

  Average F1 (micro) score: 0.8127


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV5_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.7024
  Location  : 0.7208
  Microphone: 0.7876
  Contacts  : 0.7063
  Storage   : 0.5712
  Phone     : 0.7769
  SMS       : 0.7570
  Call_Log  : 0.7419
  Calendar  : 0.8243

  Average ROC_AUC score: 0.7321
